<a href="https://colab.research.google.com/github/norhan333/NeuroPhone-RealTime-BrainMobilePhone-Interface/blob/main/model/EPFL_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!pip install -q MOABB

['\x1b[?25l',
 '\x1b[K     |██▎                             | 10 kB 4.8 MB/s eta 0:00:01',
 '\x1b[K     |████▋                           | 20 kB 2.8 MB/s eta 0:00:01',
 '\x1b[K     |███████                         | 30 kB 2.9 MB/s eta 0:00:01',
 '\x1b[K     |█████████▎                      | 40 kB 3.5 MB/s eta 0:00:01',
 '\x1b[K     |███████████▋                    | 51 kB 3.6 MB/s eta 0:00:01',
 '\x1b[K     |██████████████                  | 61 kB 3.0 MB/s eta 0:00:01',
 '\x1b[K     |████████████████▎               | 71 kB 2.7 MB/s eta 0:00:01',
 '\x1b[K     |██████████████████▋             | 81 kB 2.6 MB/s eta 0:00:01',
 '\x1b[K     |█████████████████████           | 92 kB 2.7 MB/s eta 0:00:01',
 '\x1b[K     |███████████████████████▎        | 102 kB 2.9 MB/s eta 0:00:01',
 '\x1b[K     |█████████████████████████▋      | 112 kB 2.9 MB/s eta 0:00:01',
 '\x1b[K     |████████████████████████████    | 122 kB 2.9 MB/s eta 0:00:01',
 '\x1b[K     |██████████████████████████████▎ | 133 kB 2.9 

In [119]:
import numpy as np
import scipy.signal
import scipy.io as sio
import matplotlib
from matplotlib import pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

In [3]:
import moabb
from moabb.datasets import EPFLP300, utils
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import LeftRightImagery
from moabb.pipelines.features import LogVariance
from mne.filter import construct_iir_filter
moabb.set_log_level("info")


In [4]:
def getData(subjectNumbers,dataset=EPFLP300()):
  return dataset.get_data(subjectNumbers)

In [5]:
def getSubjectData(subjectNumber,session_number,run_number):
  session = f'session_{session_number}'
  run = f'run_{run_number}'
  return getData([subjectNumber])[subjectNumber][session][run]

In [6]:
def filterData(data,lp=12,hp=1,sf=2048,resample=32):
  iir_params = dict(order=4, ftype='butter', output='sos')  
  iir_params = construct_iir_filter(iir_params, [hp,lp], None, sf, 'bandpass', return_copy=False)
  data.filter(method='iir',iir_params=iir_params,h_freq=hp,l_freq=lp)
  return data

In [7]:
lp=12;hp=1;sf=2048;resample=32
iir_params = dict(order=4, ftype='butter', output='sos')  
iir_params = construct_iir_filter(iir_params, [hp,lp], None, sf, 'bandpass', return_copy=False)
data=getSubjectData(1,1,1)
data
# x=data.filter(method='iir',iir_params=iir_params,h_freq=hp,l_freq=lp)
# x

/usr/local/lib/python3.8/dist-packages/moabb/datasets/download.py:53: RuntimeWarning: Setting non-standard config type: "MNE_DATASETS_EPFLP300_PATH"
  set_config(key, get_config("MNE_DATA"))


MNE_DATA is not already configured. It will be set to default location in the home directory - /root/mne_data
All datasets will be downloaded to this location, if anything is already downloaded, please move manually to this location


100%|████████████████████████████████████████| 267M/267M [00:00<00:00, 157GB/s]
SHA256 hash of downloaded file: a493408349f2b5fc1a94f0bb5ed3f6cdcb2cdb290af7cb4c118cbc65c7ee1128
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


unzip /root/mne_data/MNE-epflp300-data/groups/m/mm/mmspg/www/BCI/p300/subject1.zip


<RawArray | 35 x 114687 (56.0 s), ~30.7 MB, data loaded>

In [19]:
subjects={}
for subjectNumber in range(1,9):
  sessions={}
  for sessionNumber in range(1,5):
    runs={}
    for runNumber in range(1,7):
      data = getSubjectData(subjectNumber,sessionNumber,runNumber)
      print(data)
      data = filterData(data)
      print(data)
      runs[runNumber] = data.resample(32)
      print(runs[runNumber])
    sessions[sessionNumber]=runs
  subjects[subjectNumber]=sessions

<RawArray | 35 x 114687 (56.0 s), ~30.7 MB, data loaded>
<RawArray | 35 x 114687 (56.0 s), ~30.7 MB, data loaded>
<RawArray | 35 x 1792 (56.0 s), ~541 kB, data loaded>


In [9]:
def bandPassFilter(data, sampligFrequency, lowFrequency, highFrequency, order):
  sos=scipy.signal.butter(order, Wn=[lowFrequency,highFrequency],btype='bandpass', fs=sampligFrequency, output='sos')
  filteredData = scipy.signal.sosfilt(sos, data)
 #plotting
  t = np.linspace(0, 1, 114687 , False)  # 1 second
  fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
  ax1.plot(t, filteredData)
  ax1.set_title('10 Hz and 20 Hz sinusoids')
  ax1.axis([0, 1,1, 20])

  ax2.plot(t, filteredData)
  ax2.set_title('After 15 Hz high-pass filter')
  ax2.axis([0, 1,1, 20])
  ax2.set_xlabel('Time [seconds]')
  plt.tight_layout()
  plt.show()
  return filteredData

In [179]:
df = subjects[1][1][1].to_data_frame()
df.head()

,time,Fp1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,...,FC2,F4,F8,AF4,Fp2,Fz,Cz,MA1,MA2,STI
0,0.00000,0.000695,0.005022,0.010937,0.000722,-0.006139,0.009083,0.004176,0.000725,0.015054,...,-0.011320,-0.008231,0.029393,-0.013295,0.003317,-0.007605,0.003051,0.006680,-0.006680,0.0
1,0.03125,1.721186,1.962593,0.244127,2.630021,6.287801,12.691410,3.095202,8.093628,-6.216290,...,0.901251,3.852332,-7.232220,3.423578,3.188274,1.634911,2.264884,0.006667,-0.006667,0.0
2,0.06250,3.122874,2.398689,0.720184,3.597990,8.500251,14.986791,8.393488,9.389298,-8.420373,...,-0.780711,0.310387,-20.997922,3.705587,1.565788,4.075645,2.900440,0.006672,-0.006672,0.0
3,0.09375,1.914845,0.020576,0.209722,-0.144635,3.516894,4.884602,9.688516,3.144019,-5.922161,...,-5.292389,-4.037025,-16.890500,0.544083,0.727026,0.317413,3.553329,0.006671,-0.006671,0.0
4,0.12500,-0.232466,-3.972272,-0.302832,-5.573326,-2.391631,1.293344,6.990735,0.081583,-5.379548,...,-6.467250,-3.905312,-2.239258,-1.778435,-0.820241,-6.466986,1.488497,0.006676,-0.006676,0.0


In [180]:
values = df.loc[0].values
values

array([ 0.        ,  0.00069461,  0.00502216,  0.01093695,  0.00072234,
       -0.00613907,  0.0090826 ,  0.00417584,  0.00072545,  0.01505445,
        0.02245657,  0.00333073,  0.00179691, -0.00776365, -0.0067918 ,
       -0.00796106, -0.00375038, -0.00753582, -0.00966181, -0.01317375,
       -0.00335871, -0.02691801, -0.02515232, -0.00914003, -0.02298309,
       -0.00967572, -0.01131983, -0.0082305 ,  0.02939334, -0.01329538,
        0.00331689, -0.00760492,  0.00305117,  0.00667997, -0.00667997,
        0.        ])

In [181]:
def scaleSubjectData(subjectData):
    scaler = MinMaxScaler(feature_range = (-1,1))
    df = subjects[1][1][1].to_data_frame()
    columns = ['time', 'Fp1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1',
       'CP5', 'P7', 'P3', 'Pz', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8',
       'CP6', 'CP2', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'AF4', 'Fp2', 'Fz',
       'Cz', 'MA1', 'MA2', 'STI']
    time = df['time']
    df.drop(columns = ['time'],axis=1,inplace=True)
    for entry in range(len(df)):
       values = df.loc[entry].values
       values = values.reshape(35,1)
       scaledRow = scaler.fit_transform(values)
       df.loc[entry] = scaledRow.reshape(35,)
    df['time'] = time
    df = df[columns]
    return df

In [182]:
scaledDataFrame = scaleSubjectData(subjects[1][1][1])
scaledDataFrame.head()

,time,Fp1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,...,FC2,F4,F8,AF4,Fp2,Fz,Cz,MA1,MA2,STI
0,0.00000,-0.019288,0.134413,0.344488,-0.018303,-0.261998,0.278627,0.104354,-0.018192,0.490728,...,-0.446003,-0.336279,1.000000,-0.516168,0.073847,-0.314061,0.064410,0.193293,-0.281209,-0.043958
1,0.03125,0.092554,0.112523,-0.029627,0.167732,0.470300,1.000000,0.206211,0.619676,-0.564027,...,0.024729,0.268840,-0.648063,0.233374,0.213910,0.085417,0.137528,-0.049270,-0.050373,-0.049821
2,0.06250,0.363350,0.324488,0.234415,0.388846,0.651915,1.000000,0.646185,0.699623,-0.256092,...,0.153873,0.212424,-0.931037,0.394620,0.279793,0.414478,0.351414,0.196126,0.195410,0.195768
3,0.09375,0.415052,0.272513,0.286746,0.260082,0.535602,0.638518,1.000000,0.507544,-0.174662,...,-0.127273,-0.032810,-1.000000,0.311906,0.325672,0.294849,0.538344,0.271467,0.270463,0.270965
4,0.12500,0.076277,-0.359833,0.068072,-0.546537,-0.175510,0.254207,0.918597,0.112899,-0.523940,...,-0.650781,-0.352025,-0.157741,-0.104003,0.007735,-0.650750,0.276964,0.104164,0.102607,0.103386


In [183]:
scaledValues = scaledDataFrame.loc[0].values
scaledValues

array([ 0.        , -0.01928762,  0.13441306,  0.34448756, -0.01830266,
       -0.26199811,  0.27862707,  0.10435444, -0.01819243,  0.49072813,
        0.75362794,  0.07433878,  0.01986257, -0.31969799, -0.28518117,
       -0.3267097 , -0.17715951, -0.31160626, -0.38711467, -0.51184787,
       -0.16324864, -1.        , -0.93728824, -0.36858281, -0.86024408,
       -0.38760887, -0.44600251, -0.33627923,  1.        , -0.51616768,
        0.07384732, -0.3140607 ,  0.06440982,  0.19329332, -0.28120932,
       -0.043958  ])